In [23]:
# From: https://www.kaggle.com/c/dog-breed-identification/data
# Author: Morpheus Hsieh

from __future__ import print_function, division

import os, sys
import copy
import io
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir
from os.path import join, isfile
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import datasets, models, transforms, utils

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.5.1
Torchvision Version:  0.6.1


In [5]:
ProcPath = r'D:\GitWork\dog_breed\data\processed'
print('Proc path:', ProcPath)

TestPath = r'D:\GitWork\dog_breed\data\raw\test'
print("Test path: '{}'".format(TestPath))

ModelPath = r'D:\GitWork\dog_breed\models'
print("Model path: '{}'".format(ModelPath))

BestModel = '20200916-160223_resnet50_acc94.pth'

BatchSize = 6
NumClasses = 16

Proc path: D:\GitWork\dog_breed\data\processed
Test path: 'D:\GitWork\dog_breed\data\raw\test'
Model path: 'D:\GitWork\dog_breed\models'


In [6]:
# Read breed information from csv
CsvBreedsProc = 'breeds_processed.csv'
f_abspath = join(ProcPath, CsvBreedsProc)

df_breeds = pd.read_csv(f_abspath)

print(df_breeds.info())
print(); print(df_breeds.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   breed_id  120 non-null    int64 
 1   breed     120 non-null    object
 2   count     120 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.9+ KB
None

   breed_id                 breed  count
0         0    scottish_deerhound    126
1         1           maltese_dog    117
2         2          afghan_hound    116
3         3           entlebucher    115
4         4  bernese_mountain_dog    114


In [7]:
# Get most popular breeds

def getMostPopularBreeds(df, numClasses=16):
    df1 = df.sort_values(['count', 'breed'], ascending=(False, True))
    df1 = df1.head(numClasses)
    return df1

df_breeds_selected = getMostPopularBreeds(df_breeds, NumClasses)

selected_brds = list(df_breeds_selected['breed'])
print('\nSelected breeds: [\n  {}\n]'.format('\n  '.join(selected_brds)))

selected_bids = list(df_breeds_selected['breed_id'])
print('\nSelected breed ids:\n  {}'.format(selected_bids))


Selected breeds: [
  scottish_deerhound
  maltese_dog
  afghan_hound
  entlebucher
  bernese_mountain_dog
  shih-tzu
  great_pyrenees
  pomeranian
  basenji
  samoyed
  airedale
  tibetan_terrier
  cairn
  leonberg
  beagle
  japanese_spaniel
]

Selected breed ids:
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 12, 14, 15]


In [9]:
# Build breed dictionaries

def df2dict(df):
    dic = {}
    for i, row in df.iterrows():
        dic[row['breed']] = row['breed_id']
    return dic

dict_breed_fw = df2dict(df_breeds_selected)

print('Breeds dict forward:'); 
print(json.dumps(dict_breed_fw, indent=2))

Breeds dict forward:
{
  "scottish_deerhound": 0,
  "maltese_dog": 1,
  "afghan_hound": 2,
  "entlebucher": 3,
  "bernese_mountain_dog": 4,
  "shih-tzu": 5,
  "great_pyrenees": 6,
  "pomeranian": 7,
  "basenji": 8,
  "samoyed": 9,
  "airedale": 10,
  "tibetan_terrier": 11,
  "cairn": 13,
  "leonberg": 12,
  "beagle": 14,
  "japanese_spaniel": 15
}


In [30]:
# Selected labels

CsvLabelsProc = 'labels_processed.csv'
f_abspath = join(ProcPath, CsvLabelsProc)

df_labels = pd.read_csv(f_abspath)
print('\nOrigin labels:')
print(df_labels.info())

df_labels_selected = df_labels[df_labels['breed_id'].isin(selected_bids)]

def newInfoStr(df, indent=4):
    buf = io.StringIO()
    df.info(buf=buf)
    pad_str = (' ' * indent)
    old_str = '\n'
    new_str = '\n' + pad_str
    outstr = buf.getvalue().replace(old_str, new_str)
    return pad_str + outstr

outstr = newInfoStr(df_labels_selected)
print('\nSelected labels:\n')
print(outstr)

print(df_labels_selected.head())


Origin labels:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10222 entries, 0 to 10221
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image     10222 non-null  object
 1   breed_id  10222 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 159.8+ KB
None

Selected labels:

    <class 'pandas.core.frame.DataFrame'>
    Int64Index: 1777 entries, 8 to 10219
    Data columns (total 2 columns):
     #   Column    Non-Null Count  Dtype 
    ---  ------    --------------  ----- 
     0   image     1777 non-null   object
     1   breed_id  1777 non-null   int64 
    dtypes: int64(1), object(1)
    memory usage: 41.6+ KB
    
                                                image  breed_id
8   D:\GitWork\dog_breed\data\raw\train\003df8b8a8...         8
9   D:\GitWork\dog_breed\data\raw\train\0042188c89...         0
12  D:\GitWork\dog_breed\data\raw\train\00693b8bc2...         1
29  D:\GitWork\dog_breed\data\raw\train\0

In [41]:
# Dataset

normalize = transforms.Normalize(
    mean = [0.485, 0.456, 0.406],
    std  = [0.229, 0.224, 0.225]
)

# Transform
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

class myDataset(Dataset):

    def __init__(self, df_selected, transform=None):

        self.images = list(df_selected['image'])
        self.labels = list(df_selected['breed_id'])
        self.len = len(df_selected)
        self.transform = transform

    def __getitem__(self, index):
        image = self.images[index]
        img_pil = Image.open(image)

        if self.transform is not None:
            img = self.transform(img_pil)

        lbl = int(self.labels[index])
        
        return [img, lbl]

    def __len__(self):
        return self.len
    
dataSet = myDataset(df_labels_selected, transform)
dataLoader = DataLoader(dataSet, batch_size=BatchSize, shuffle=False)
datasize = { 'test': len(dataLoader) }

In [42]:
inputs, labels = next(iter(dataLoader))
print('\nImage type:', type(inputs,))
print('      size: ', inputs.size())

print('\nLabel type:', type(labels))
print('      size: ', labels.size())

inp = inputs[0]
print('\nImage shape:', inp.shape)
print(); print(inp)

print('\nLabels:', labels)


Image type: <class 'torch.Tensor'>
      size:  torch.Size([6, 3, 224, 224])

Label type: <class 'torch.Tensor'>
      size:  torch.Size([6])

Image shape: torch.Size([3, 224, 224])

tensor([[[ 0.0912,  0.0569,  0.2453,  ...,  0.1426, -0.1486, -0.5082],
         [ 0.2624,  0.2624,  0.3652,  ...,  0.3309, -0.2513, -0.5767],
         [ 0.6734,  0.5193,  0.1083,  ...,  0.4166,  0.4337,  0.0398],
         ...,
         [-0.1143, -0.0801,  0.1083,  ...,  1.5468,  0.2453, -0.1486],
         [-0.4226, -0.1486,  0.3652,  ...,  1.6495,  1.0673, -0.1143],
         [-0.6623,  0.0056,  0.0569,  ...,  0.2796,  0.2967,  0.3823]],

        [[ 0.3627,  0.3627,  0.5728,  ...,  0.3978,  0.1702, -0.1450],
         [ 0.4853,  0.5203,  0.6954,  ...,  0.5553,  0.0301, -0.2500],
         [ 0.8179,  0.7304,  0.4153,  ...,  0.6254,  0.6779,  0.3102],
         ...,
         [ 0.3978,  0.4153,  0.5728,  ...,  1.6583,  0.3102, -0.0749],
         [ 0.0651,  0.2752,  0.8004,  ...,  1.6057,  0.9755, -0.1625],
     

In [7]:
# prediction
use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if use_gpu else "cpu")
print(device)

resnet = models.resnet50(pretrained=True)

PreTrainedModel = 'resnet50-19c8e357.pth'
model_abspath = join(ModelPath, PreTrainedModel)
resnet.load_state_dict(torch.load(model_abspath))

# for param in resnet.parameters():
#     param.requires_grad = False

num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, NUM_BREED_CLASSES)

resnet = resnet.cuda()

df = pd.DataFrame(columns=['id', 'breed'])

datapath = TestPath
id_list = [
    f.replace('.jpg', '') for f in listdir(datapath) \
    if f.endswith('.jpg') and isfile(join(datapath, f))
]

num = 10
for i in range(len(id_list[:num])):
    img_id = id_list[i]
    img_path = join(TestPath, img_id+'.jpg')
    img_pil = Image.open(img_path)
    img_tensor = transform(img_pil).unsqueeze_(0)
    inp = Variable(img_tensor)
    inp = inp.to(device)
    res = resnet(inp)
    index = res.data.cpu().numpy().argmax()
   
    df.loc[len(df)] = [img_id, index]


print()
print(df.info())
print()
print(df.head(num))


cuda:0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10 non-null     object
 1   breed   10 non-null     object
dtypes: object(2)
memory usage: 240.0+ bytes
None

                                 id breed
0  000621fb3cbb32d8935728e48679680e    10
1  00102ee9d8eb90812350685311fe5890    10
2  0012a730dfa437f5f3613fb75efcd4ce    10
3  001510bc8570bbeee98c8d80c8a95ec1    10
4  001a5f3114548acdefa3d4da05474c2e    10
5  00225dcd3e4d2410dd53239f95c0352f    10
6  002c2a3117c2193b4d26400ce431eebd    10
7  002c58d413a521ae8d1a5daeb35fc803    10
8  002f80396f1e3db687c5932d7978b196    10
9  0036c6bcec6031be9e62a257b1c3c442    10
